In [1]:
#Import libraries.
import pandas as pd
import rapidfuzz
from nltk.corpus import stopwords
import numpy as np
from yake import KeywordExtractor

In [2]:
Stops = stopwords.words('english') #Returns all stopwords in the English language. 
print(Stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
# final manual dict
datadict = {'adm':'administrator', 'administer':'administrator', 'administratrix estate joshua brown':'administrator',
            'adminastrator':'administrator',
 'ah.vat':'lawyer', 'asse john b':'executor', 'behalf':'executor', 'black':'blacksmith',
 'blacks':'blacksmith', 'book':'book binder', 'chaise':'chaise maker', 'charleston':'merchant',
 'christopher':'executor', 'cordwainder':'cordwainer', 'esq':'esquire', 'esquier':'esquire',
 'ex':'executor', 'excuetor':'executor', 'executers':'executor', 'executor michael de mott deceased':'executor', 'executrix':'executor',
 'extors late jn jenckes': 'executor', 'extors late nicholas brown':'executor','frderick':'soldier',
 'general treasurer state rhode island suceessors':'treasurer','hair':'hair dresser','in trust jacobus van eis somerset county new jersey':'executor',
 'inn':'inn keeper', 'innholder':'inn keeper', 'law':'lawyer', 'leather':'leather dresser', 'penniston': np.nan, 'pennsylvania': np.nan, 'post':'post master',
 'property':'executor','school':'school master', 'ship':'ship wright','shop':'shop keeper', 'society':'treasurer', 'south':np.nan,
 'the state':np.nan,'vendue':'vendue master', 'treasurer baptist benevolent':'treasurer', 'treasurer charitable baptist':'treasurer',
 'treasurer congregational benevolent':'treasurer', 'treasurer society cincinnati state':'treasurer', 'treasurer st johns ledge no 1 tree accepted':'treasurer',
 'treasurer state rhode island':'treasurer', 'tresurer catholic':'treasurer', 'trust':'trustee','baird':'executor',
            'cabinet':'cabinet maker', 'asse john b ashe exe' : 'executors', 'trust' : 'assignee', 'pennsylvania' : 'no occupation', 'co' : 'no occupation', 'penniston' : 'no occupation',
'committee':'committee member', 'controller':'customs controller',
            'occupation':'no occupation'
 }

# Conneticut

In [4]:
State = 'CT'
AllOccs = pd.read_csv('../../../Data/Post1790/Aggregated/aggregated_CD.csv') #Returns aggregated CSV file.
AllOccs.loc[AllOccs[AllOccs['occupation'].apply(lambda x: pd.isnull(x))].index, 'occupation'] = 'No Occupation'
AllOccs.drop(columns=['Unnamed: 0'], inplace=True) #Drop duplicate index column which appears.
AllOccsCT = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsCT = AllOccsCT.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsCT)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       1064.0      75.0          532.0          37.0      508.0      51.0   
1        449.0      96.0          224.0          97.0      232.0      10.0   
2        196.0      75.0           98.0          37.0      172.0      24.0   
3         53.0      58.0           26.0          79.0       67.0       6.0   
4        345.0      50.0          172.0          75.0      248.0      76.0   
..         ...       ...            ...           ...        ...       ...   
910       39.0      11.0           19.0          55.0       14.0       7.0   
911       13.0      21.0            6.0          61.0        4.0      74.0   
912      533.0      34.0          266.0          66.0      446.0      66.0   
913      658.0      47.0          329.0          23.0      558.0      50.0   
914       12.0      22.0            6.0          11.0       11.0       NaN   

             town state     occupation                   Name  

In [5]:
OccsTotalsCT = AllOccsCT.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsCT['# of people'] = OccsTotalsCT['occupation'].map(AllOccsCT['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
ogoccs = OccsTotalsCT[['occupation']]
print(OccsTotalsCT.to_string())

                                                           occupation   6p_total  # of people
0                                   Adminastrator of the Asa Waterman     218.47            1
1                                      Adminastrator to David Fellows       0.00            1
2   Adminastrator to the Estate of Epaphras Bull of Hartford Deceased      61.30            1
3                        Adminastrator to the Estate of Samuel Tilley      94.67            1
4            Adminastrator to the estate of Uriah Loomis Jun Deceased      73.50            1
5                                          Administrator to J McCurdy    3413.72            2
6                         Administrator to the estate of Daniel Unkus      54.05            1
7                Adminstrator to the Estate of Ezekiel Carey Deceased      77.61            1
8                  Adminstrator to the Estate of Ruth Skiffe Deceased      87.74            1
9                                                          A

In [6]:
#Function that returns the same dataframe, but with all stopwords removed from the occupation titles. 
def RemoveStopwords(OccsTotals):
    OccsTotals['occupation'] = OccsTotals['occupation'].apply(lambda OccName: ' '.join(word.lower() for word in OccName.split() if word not in Stops))
    return OccsTotals 

In [7]:
OccsTotalsCT = RemoveStopwords(OccsTotalsCT) #Call 
print(OccsTotalsCT.to_string())
ogoccs['new occupation'] = OccsTotalsCT['occupation']

                                              occupation   6p_total  # of people
0                             adminastrator asa waterman     218.47            1
1                            adminastrator david fellows       0.00            1
2   adminastrator estate epaphras bull hartford deceased      61.30            1
3                     adminastrator estate samuel tilley      94.67            1
4         adminastrator estate uriah loomis jun deceased      73.50            1
5                                administrator j mccurdy    3413.72            2
6                      administrator estate daniel unkus      54.05            1
7             adminstrator estate ezekiel carey deceased      77.61            1
8               adminstrator estate ruth skiffe deceased      87.74            1
9                                             apothecary       7.72            1
10                                              attorney    4863.67            3
11                          

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/1356372720.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ogoccs['new occupation'] = OccsTotalsCT['occupation']


In [8]:
Remove = 2 #Define remove constant. 

In [9]:
#This function will help determine how many words we can acceptably cut off from the end. 
def FindRemoveConst(Occupations):
    TempMaxLength = 2
    
    print('Occupation Before --> Occupation After')
    print('--------------------------------------')
    print('\n')

    retdict = dict()

    for Occupation in Occupations:
        if (len(Occupation.split()) > TempMaxLength): #If occupation has more than two words, remove last two words.
            OccupationAfter = Occupation.rsplit(' ', Remove)[0] 
        else:
            OccupationAfter = Occupation #Else, set them both equal to each other; nothing changes. 
        retdict[Occupation] = OccupationAfter
        print(Occupation + ' --> ' + OccupationAfter)
    return retdict

In [10]:
retdict = FindRemoveConst(OccsTotalsCT['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict[x])

Occupation Before --> Occupation After
--------------------------------------


adminastrator asa waterman --> adminastrator
adminastrator david fellows --> adminastrator
adminastrator estate epaphras bull hartford deceased --> adminastrator estate epaphras bull
adminastrator estate samuel tilley --> adminastrator estate
adminastrator estate uriah loomis jun deceased --> adminastrator estate uriah loomis
administrator j mccurdy --> administrator
administrator estate daniel unkus --> administrator estate
adminstrator estate ezekiel carey deceased --> adminstrator estate ezekiel
adminstrator estate ruth skiffe deceased --> adminstrator estate ruth
apothecary --> apothecary
attorney --> attorney
attorney law --> attorney law
blacksmith --> blacksmith
broker --> broker
carpenter --> carpenter
cashier hartford bank --> cashier
clothier --> clothier
committee 1st ecclestical society --> committee 1st
committee 1st society plainville --> committee 1st
cooper --> cooper
doctor --> doctor
esqui

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/3165089881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict[x])


In [11]:
#Declare necessary parameters for KeywordExtractor() function.
MaxLength = 2
language = 'en' #Set language to English.
MaxSize = 1  #Each keyword can only be a max. length of 1.
DupThres = 0.5 #Words can not be used repeatedly in keywords. Treat each word separately. 
KeyExtractor = KeywordExtractor(lan=language, n=MaxSize, dedupLim=DupThres, features=None) #Create KeywordExtractor object.
NoMerge = []

In [12]:
# Manually clean items. 
Exceptions = {'inn keeper': 'inn keeper', 'leather dresser': 'leather dresser', 'post rider': 'post rider', 
              'school committee': 'school committee', 'school master': 'school master', 'society committee': 'committee', 
              'society treasurer': 'treasurer', 'frederick company': 'frederick company', 'the state north carolina': None,
              'the state of north carolina' : 'the state of north carolina','in trust jacobus van eis somerset county': 'assignee', 
              'ah.vat law': 'attorney', 'black smith': 'blacksmith','book binder': 'book binder', 'ex estate john baird': 'executors', 
              'ex': 'executors', 'junior broker': 'broker', 'post master': 'post master', 'ship wright': 'ship wright', 'shop keeper': 'shopkeeper', 
              'christopher mary': 'individual', 'law': 'attorney', 'lawyer': 'attorney', 
              'adm alexander mccaraher late chester': 'administer', 
              'general treasurer state rhode island suceessors said': 'treasurer', 'behalf proprietors': 'proprietors', 
              'chaise maker': 'chaise maker', 'executrix william': 'executor', 'blacks smith': 'blacksmith', 
              'cabinet maker': 'cabinet maker', 'charleston merchants': 'merchants', 'hair dresser': 'hair dresser', 
              'south carolina': 'south carolina', 'vendue master': 'auctioneer', 'physician' : 'doctor', 'administer' : 'adminastrator',
                'esquier' : 'esq', 'excuters' : 'excuetor', 'extors' : 'excuetor', 'esquire' : 'esq', 'executers' : 'excuetor',
             'yeoman' : 'farmer', 'cordwainer' : 'shoemaker', 'joiner' : 'carpenter', 'innholder' : 'inn keeper', 'factor' : 'trader',
             'asse john b ashe exe' : 'executors', 'trust' : 'assignee', 'pennsylvania' : None, 'co' : None, 'penniston' : None,
              'committee':'committee member', 'controller':'customs controller'}

In [13]:
#Print alphabetized version of 'Exceptions'.
for Name in sorted(Exceptions):
    print("%s: %s" % (Name, Exceptions[Name]))

adm alexander mccaraher late chester: administer
administer: adminastrator
ah.vat law: attorney
asse john b ashe exe: executors
behalf proprietors: proprietors
black smith: blacksmith
blacks smith: blacksmith
book binder: book binder
cabinet maker: cabinet maker
chaise maker: chaise maker
charleston merchants: merchants
christopher mary: individual
co: None
committee: committee member
controller: customs controller
cordwainer: shoemaker
esquier: esq
esquire: esq
ex: executors
ex estate john baird: executors
excuters: excuetor
executers: excuetor
executrix william: executor
extors: excuetor
factor: trader
frederick company: frederick company
general treasurer state rhode island suceessors said: treasurer
hair dresser: hair dresser
in trust jacobus van eis somerset county: assignee
inn keeper: inn keeper
innholder: inn keeper
joiner: carpenter
junior broker: broker
law: attorney
lawyer: attorney
leather dresser: leather dresser
penniston: None
pennsylvania: None
physician: doctor
post ma

In [14]:
#Print what the merged occupations will look like. This will help us find any exceptions. 
def PrintFutureOccupations(Occupations):    
    print('Occupations --> Potential Cleaned Occupations')
    print('---------------------------------------------')
    print('\n')

    retdict = dict()
    for Occupation in Occupations:
        if (len(Occupation.split()) > MaxLength): #If occupation has more than two words in it, remove last two words. Depends on constant. Might change. 
            Occupation = Occupation.rsplit(' ', Remove)[0]
            
        Keywords = KeyExtractor.extract_keywords(Occupation) #Retrieve list of keywords. 
        
        if (len(Keywords) == 0):
            Keywords.append((Occupation, ' '))
        retdict[Occupation] = str(Keywords[0][0])
        print(Occupation + ' --> ' + str(Keywords[0][0])) #Print current occupation title to hypothetical cleaned occupation title.
    return retdict

In [15]:
retdict = PrintFutureOccupations(OccsTotalsCT['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict[x])

Occupations --> Potential Cleaned Occupations
---------------------------------------------


adminastrator --> adminastrator
adminastrator --> adminastrator
adminastrator estate epaphras bull --> adminastrator
adminastrator estate --> adminastrator
adminastrator estate uriah loomis --> adminastrator
administrator --> administrator
administrator estate --> administrator
adminstrator estate ezekiel --> adminstrator
adminstrator estate ruth --> adminstrator
apothecary --> apothecary
attorney --> attorney
attorney law --> attorney
blacksmith --> blacksmith
broker --> broker
carpenter --> carpenter
cashier --> cashier
clothier --> clothier
committee 1st --> committee
committee 1st --> committee
cooper --> cooper
doctor --> doctor
esquire --> esquire
executor ezra selden --> executor
executor stephen --> executor
executor stephen --> executor
executor william --> executor
executor estate tim --> executor
executor estate uriah --> executor
executors estate jabez --> executors
farmer --> farm

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/355176414.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict[x])


In [16]:
#Handle duplicate occupations that are spelled differently. Pass in 'occupation' column as a list. Pass in function which will do a manual check for each state (differs w/ each state). 
def MergeOccupations(OccsTotals):
    OccsTotals['% of total people'] = None #Add a percent of total population column.
    Occupations = OccsTotals['occupation'].tolist() #Returns 'occupation' column as a list.    
    Index = 0 #Keep track of index. 
    
    for Occupation in Occupations:
        if (len(Occupation.split()) > MaxLength): #If occupation has more than two words in it, remove last two words. Depends on constant. Might change. 
            Occupation = Occupation.rsplit(' ', Remove)[0]
            
        Keywords = KeyExtractor.extract_keywords(Occupation) #Retrieve list of keywords. 
        
        if (len(Keywords) == 0):
            Keywords.append((Occupation, ' '))
        
        NewOccupation = Keywords[0][0] #Get new occupation from list of keywords. Based on which keyword has the highest rating.
                
        if (Occupation in Exceptions): #If occupation is an exception, set new occupation. 
            NewOccupation = Exceptions[Occupation]
    
        OccsTotals.loc[Index, 'occupation'] = NewOccupation #Replace occupation with new occupation in dataframe.
        
        Index += 1 #Increment. 
    
    print('After Cleaning')
    print('--------------')
    print('\n')
    print(OccsTotals.to_string()) #Check
    print('\n')
    
    OccsTotals = OccsTotals.groupby('occupation')[['6p_total', '# of people', '% of total people']].apply(sum).reset_index() #Combine duplicate occupations, reset index. 
    OccsTotals['# of people'] = OccsTotals['# of people'].astype(int) #Convert values in '# of people' column to all ints. 
    
    print('After Removing Duplicates and Summing')
    print('-------------------------------------')
    print('\n')
    print(OccsTotals.to_string()) #Check
    print('\n')
    retdict = dict()
    Occupations = OccsTotals['occupation'].tolist() #Reinstantiate Occupations after changing OccsTotals
    i = 0 #Declare new index.
    while (i < len(Occupations)): 
        j = 0 #Declare new index.
        while (j < len(Occupations)):                
            if (rapidfuzz.fuzz.ratio(Occupations[i], Occupations[j]) > 80) and (Occupations[i] != Occupations[j]) and ((Occupations[j] not in NoMerge) and (Occupations[i] not in NoMerge))  : #Use fuzzy string matching to determine whether two occupations are the same or not.               
                Total = float(OccsTotals.loc[j, '6p_total']) #Get total from duplicate occupation.
                OccsTotals.loc[i, '6p_total'] += Total #Merge totals together.
                
                NumOfPeople = OccsTotals.loc[j, '# of people'] #Get number of people in row j.
                OccsTotals.loc[i, '# of people'] += NumOfPeople #Add that number to row i.
                
                OccsTotals.drop(j, inplace=True) #Remove duplicate occupation.
                OccsTotals = OccsTotals.reset_index(drop=True) #Reset index.

                retdict[Occupations[j]] = Occupations[i]
                print('Removed: ' + Occupations[j] + ' --> ' + Occupations[i])

                Occupations.remove(Occupations[j]) #Remove from list of occupations so loop will not iterate over it again.
            else:
                j += 1
                
        i += 1
        
    OccsTotals['average debt per person'] = OccsTotals['6p_total'] / OccsTotals['# of people'] #Add an average column.
    OccsTotals['% of total people'] = (OccsTotals['# of people'] / OccsTotals['# of people'].sum()) * 100 #Assign values to '# of people column'.
    OccsTotals = OccsTotals.sort_values('6p_total', ascending=False).reset_index(drop=True)
    
    print('\n')
    print('After Using Fuzzy String Matching')
    print('---------------------------------')
    print('\n')
    print(OccsTotals.to_string())
    print('\n')
    
    return retdict, OccsTotals
        

In [17]:
retdict, OccsTotalsCT = MergeOccupations(OccsTotalsCT)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


          occupation   6p_total  # of people % of total people
0      adminastrator     218.47            1              None
1      adminastrator       0.00            1              None
2      adminastrator      61.30            1              None
3      adminastrator      94.67            1              None
4      adminastrator      73.50            1              None
5      administrator    3413.72            2              None
6      administrator      54.05            1              None
7       adminstrator      77.61            1              None
8       adminstrator      87.74            1              None
9         apothecary       7.72            1              None
10          attorney    4863.67            3              None
11          attorney    6250.93           17              None
12        blacksmith     724.71            3              None
13            broker     208.55            3              None
14         carpenter   

In [18]:
def TurnToExcel(DF, State):
    DF['occupation'] = DF['occupation'].apply(lambda x: datadict.get(x, x))
    DF.to_excel('./FinishedSpreadsheets/Occupations' + State + '.xlsx')

In [19]:
TurnToExcel(OccsTotalsCT, State)

# Georgia

In [20]:
State = 'GA'
AllOccsGA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsGA = AllOccsGA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsGA)

    6p_Dollar    6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0      4586.0     66.0000         2293.0       33.0000     3026.0     11.00   
1         1.0     95.1133            1.0       47.5567        NaN      0.00   
2         1.0    890.0000            1.0      445.0000        1.0    570.70   
3         1.0     90.3000            1.0       45.1000        1.0     93.50   
4         1.0    249.7700            1.0      124.8900        1.0    273.65   
5         1.0    114.6000            1.0       57.3000        1.0    123.94   
6         1.0    227.5133            1.0      113.7533        NaN       NaN   
7         1.0    129.6533            1.0       64.8233        1.0    144.96   
8         1.0    209.8633            1.0      104.9333        1.0    241.38   
9         1.0  10108.9033            1.0     5054.4533        1.0   2729.40   
10        1.0    114.7330            1.0       57.3330        1.0    124.45   
11       17.0    107.8000           13.0       53.90

In [21]:
OccsTotalsGA = AllOccsGA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsGA['# of people'] = OccsTotalsGA['occupation'].map(AllOccsGA['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsGA.to_string())

temp_df = OccsTotalsGA[['occupation']]

      occupation     6p_total  # of people
0            Esq   102.089033            1
1      Gentleman  4586.660000            1
2       Merchant    29.361296            5
3      Merchants    39.478533            2
4  No Occupation   373.018899           11


In [22]:
OccsTotalsGA = RemoveStopwords(OccsTotalsGA) 
print(OccsTotalsGA.to_string())

temp_df['new occupation'] = OccsTotalsGA['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

      occupation     6p_total  # of people
0            esq   102.089033            1
1      gentleman  4586.660000            1
2       merchant    29.361296            5
3      merchants    39.478533            2
4  no occupation   373.018899           11


/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/925087446.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsGA['occupation']


In [23]:
retdict = FindRemoveConst(OccsTotalsGA['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


esq --> esq
gentleman --> gentleman
merchant --> merchant
merchants --> merchants
no occupation --> no occupation


In [24]:
retdict, OccsTotalsGA = MergeOccupations(OccsTotalsGA)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


   occupation     6p_total  # of people % of total people
0         esq   102.089033            1              None
1   gentleman  4586.660000            1              None
2    merchant    29.361296            5              None
3   merchants    39.478533            2              None
4  occupation   373.018899           11              None


After Removing Duplicates and Summing
-------------------------------------


   occupation     6p_total  # of people  % of total people
0         esq   102.089033            1                  0
1   gentleman  4586.660000            1                  0
2    merchant    29.361296            5                  0
3   merchants    39.478533            2                  0
4  occupation   373.018899           11                  0


Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


   occupation     6p_total  # of people  % of total people  average debt per per

In [25]:
TurnToExcel(OccsTotalsGA, State)

# Maryland

In [26]:
State = 'MD'
AllOccsMD = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsMD = AllOccsMD.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsMD)


     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0          1.0   58.7600            1.0       29.3800        1.0     64.17   
1         17.0   66.5733           13.0       33.2833       14.0     76.40   
2      26745.0   77.0000        13372.0       88.0000    50000.0     51.00   
3       2049.0   84.0000         1024.0       91.0000     1470.0     48.00   
4       1215.0   45.0000          607.0       72.0000     1292.0     90.00   
..         ...       ...            ...           ...        ...       ...   
517      303.0   12.0000          151.0       56.0000      275.0     32.00   
518      587.0   63.0000          293.0       82.0000      437.0     41.00   
519      283.0   33.0000          141.0       66.0000      261.0     59.00   
520       33.0   33.0000           16.0       67.0000       31.0     67.00   
521      124.0   33.0000           62.0       17.0000      118.0     41.00   

                      town state     occupation  \
0           

In [27]:
OccsTotalsMD = AllOccsMD.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsMD['# of people'] = OccsTotalsMD['occupation'].map(AllOccsMD['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsMD.to_string())

temp_df = OccsTotalsMD[['occupation']]

                         occupation       6p_total  # of people
0     Administrator of John Johnson      22.500000            1
1   Adminstrator of Robert Whitaker       0.000000            1
2                               Esq      17.665733            1
3            Executor of Moses Ruth      44.840000            1
4        Executor of Sabret Sollars     141.030000            1
5        Executor of Thomas Sollars     117.900000            1
6                 Frederick Company     206.670000            1
7                          Merchant   48636.870000           45
8                         Merchants   29874.070000            2
9                     No Occupation  767788.357600          465
10                        Shoemaker    1237.070000            2
11                       Shopkeeper      33.330000            1


In [28]:
OccsTotalsMD = RemoveStopwords(OccsTotalsMD)
temp_df['new occupation'] = OccsTotalsMD['occupation']
ogoccs = pd.concat([ogoccs, temp_df])
print(OccsTotalsMD.to_string())

                      occupation       6p_total  # of people
0     administrator john johnson      22.500000            1
1   adminstrator robert whitaker       0.000000            1
2                            esq      17.665733            1
3            executor moses ruth      44.840000            1
4        executor sabret sollars     141.030000            1
5        executor thomas sollars     117.900000            1
6              frederick company     206.670000            1
7                       merchant   48636.870000           45
8                      merchants   29874.070000            2
9                  no occupation  767788.357600          465
10                     shoemaker    1237.070000            2
11                    shopkeeper      33.330000            1


/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/646621115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsMD['occupation']


In [29]:
retdict = FindRemoveConst(OccsTotalsMD['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


administrator john johnson --> administrator
adminstrator robert whitaker --> adminstrator
esq --> esq
executor moses ruth --> executor
executor sabret sollars --> executor
executor thomas sollars --> executor
frederick company --> frederick company
merchant --> merchant
merchants --> merchants
no occupation --> no occupation
shoemaker --> shoemaker
shopkeeper --> shopkeeper


In [30]:
retdict = PrintFutureOccupations(OccsTotalsMD['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administrator --> administrator
adminstrator --> adminstrator
esq --> esq
executor --> executor
executor --> executor
executor --> executor
frederick company --> frederick
merchant --> merchant
merchants --> merchants
no occupation --> occupation
shoemaker --> shoemaker
shopkeeper --> shopkeeper


In [31]:
retdict, OccsTotalsMD = MergeOccupations(OccsTotalsMD)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


           occupation       6p_total  # of people % of total people
0       administrator      22.500000            1              None
1        adminstrator       0.000000            1              None
2                 esq      17.665733            1              None
3            executor      44.840000            1              None
4            executor     141.030000            1              None
5            executor     117.900000            1              None
6   frederick company     206.670000            1              None
7            merchant   48636.870000           45              None
8           merchants   29874.070000            2              None
9          occupation  767788.357600          465              None
10          shoemaker    1237.070000            2              None
11         shopkeeper      33.330000            1              None


After Removing Duplicates and Summing
-------------------------------------


    

In [32]:
TurnToExcel(OccsTotalsMD, State)

# North Carolina

In [33]:
State = 'NC'
AllOccsNC = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNC = AllOccsNC.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNC)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0      1865.0     58.00          932.0         74.00     1950.0      61.0   
1        85.0     25.00           42.0         62.00       41.0      82.0   
2       989.0     88.00          494.0         94.00     1006.0      12.0   
3       335.0     50.00          167.0         75.00      155.0      96.0   
4       174.0     32.00           87.0         16.00        NaN       NaN   
5        53.0      2.00           26.0         51.00        NaN       NaN   
6        30.0     81.00           15.0         40.00        NaN       NaN   
7         4.0       NaN            2.0           NaN        NaN       NaN   
8         5.0     59.00            2.0         79.00        NaN       NaN   
9       116.0     76.00           58.0         38.00        NaN       NaN   
10     1330.0     16.00          665.0          8.00      873.0      78.0   
11     1577.0     20.00          788.0         60.00     1748.0      70.0   

In [34]:
OccsTotalsNC = AllOccsNC.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsNC['# of people'] = OccsTotalsNC['occupation'].map(AllOccsNC['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsNC.to_string())

temp_df = OccsTotalsNC[['occupation']]

                    occupation    6p_total  # of people
0      Asse of John B Ashe Exe      0.0000            1
1            Doctors of Physic   2008.3700            1
2                       Farmer     62.5100            1
3                     Merchant    813.0300            5
4                    Merchants     48.5000            2
5                No Occupation  13703.6567           47
6  The State of North Carolina   4780.8400            1
7                    Treasurer  12283.4400            1


In [35]:
OccsTotalsNC = RemoveStopwords(OccsTotalsNC) 
print(OccsTotalsNC.to_string())

temp_df['new occupation'] = OccsTotalsNC['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

                 occupation    6p_total  # of people
0      asse john b ashe exe      0.0000            1
1            doctors physic   2008.3700            1
2                    farmer     62.5100            1
3                  merchant    813.0300            5
4                 merchants     48.5000            2
5             no occupation  13703.6567           47
6  the state north carolina   4780.8400            1
7                 treasurer  12283.4400            1


/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/3990776363.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsNC['occupation']


In [36]:
retdict = FindRemoveConst(OccsTotalsNC['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


asse john b ashe exe --> asse john b
doctors physic --> doctors physic
farmer --> farmer
merchant --> merchant
merchants --> merchants
no occupation --> no occupation
the state north carolina --> the state
treasurer --> treasurer


In [37]:
Remove = 0

In [38]:
retdict = PrintFutureOccupations(OccsTotalsNC['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


asse john b ashe exe --> asse
doctors physic --> doctors
farmer --> farmer
merchant --> merchant
merchants --> merchants
no occupation --> occupation
the state north carolina --> carolina
treasurer --> treasurer


In [39]:
retdict, OccsTotalsNC = MergeOccupations(OccsTotalsNC)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


   occupation    6p_total  # of people % of total people
0   executors      0.0000            1              None
1     doctors   2008.3700            1              None
2      farmer     62.5100            1              None
3    merchant    813.0300            5              None
4   merchants     48.5000            2              None
5  occupation  13703.6567           47              None
6        None   4780.8400            1              None
7   treasurer  12283.4400            1              None


After Removing Duplicates and Summing
-------------------------------------


   occupation    6p_total  # of people  % of total people
0     doctors   2008.3700            1                  0
1   executors      0.0000            1                  0
2      farmer     62.5100            1                  0
3    merchant    813.0300            5                  0
4   merchants     48.5000            2                  0
5  occupation  13703.6567  

In [40]:
TurnToExcel(OccsTotalsNC, State)

# New Hampshire

In [41]:
State = 'NH'
AllOccsNH = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNH = AllOccsNH.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNH)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        371.0      45.0          185.0          72.0      141.0      71.0   
1        200.0       NaN          100.0           NaN       54.0       NaN   
2        778.0       4.0          389.0           2.0      288.0       NaN   
3        222.0      78.0          111.0          40.0        NaN       NaN   
4         68.0      72.0           34.0          36.0       73.0      24.0   
..         ...       ...            ...           ...        ...       ...   
169      187.0       2.0           93.0          51.0      146.0      70.0   
170       12.0      73.0            6.0          37.0        0.0       0.0   
171        9.0      16.0            4.0          58.0       10.0      16.0   
172      168.0      28.0           84.0          13.0       60.0      54.0   
173       40.0      67.0           20.0          33.0       25.0      96.0   

                                  town state     occupation  \


In [42]:
OccsTotalsNH = AllOccsNH.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsNH['# of people'] = OccsTotalsNH['occupation'].map(AllOccsNH['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsNH.to_string())

temp_df = OccsTotalsNH[['occupation']]

       occupation  6p_total  # of people
0           Baker   2159.04            1
1      Blacksmith     54.88            1
2           Clerk    715.60            7
3      Cordwainer    742.05            2
4         Esquire  61539.89           66
5          Farmer    778.04            1
6       Gentleman   3603.06            6
7       Goldsmith    871.32            1
8          Joiner    106.61            2
9        Marniner    785.62            2
10       Merchant  29715.41           26
11  No Occupation  48957.43            8
12        Painter     48.61            1
13      Physician   7111.43           18
14       Spinster     89.13            1
15         Trader   2807.27            8
16      Treasurer    149.62            1
17          Widow   8790.45           10
18         Yeoman   3397.94           12


In [43]:
OccsTotalsNH = RemoveStopwords(OccsTotalsNH) 
print(OccsTotalsNH.to_string())

temp_df['new occupation'] = OccsTotalsNH['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

       occupation  6p_total  # of people
0           baker   2159.04            1
1      blacksmith     54.88            1
2           clerk    715.60            7
3      cordwainer    742.05            2
4         esquire  61539.89           66
5          farmer    778.04            1
6       gentleman   3603.06            6
7       goldsmith    871.32            1
8          joiner    106.61            2
9        marniner    785.62            2
10       merchant  29715.41           26
11  no occupation  48957.43            8
12        painter     48.61            1
13      physician   7111.43           18
14       spinster     89.13            1
15         trader   2807.27            8
16      treasurer    149.62            1
17          widow   8790.45           10
18         yeoman   3397.94           12


/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/1513769828.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsNH['occupation']


In [44]:
retdict = FindRemoveConst(OccsTotalsNH['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


baker --> baker
blacksmith --> blacksmith
clerk --> clerk
cordwainer --> cordwainer
esquire --> esquire
farmer --> farmer
gentleman --> gentleman
goldsmith --> goldsmith
joiner --> joiner
marniner --> marniner
merchant --> merchant
no occupation --> no occupation
painter --> painter
physician --> physician
spinster --> spinster
trader --> trader
treasurer --> treasurer
widow --> widow
yeoman --> yeoman


In [45]:
retdict = PrintFutureOccupations(OccsTotalsNH['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


baker --> baker
blacksmith --> blacksmith
clerk --> clerk
cordwainer --> cordwainer
esquire --> esquire
farmer --> farmer
gentleman --> gentleman
goldsmith --> goldsmith
joiner --> joiner
marniner --> marniner
merchant --> merchant
no occupation --> occupation
painter --> painter
physician --> physician
spinster --> spinster
trader --> trader
treasurer --> treasurer
widow --> widow
yeoman --> yeoman


In [46]:
retdict, OccsTotalsNH = MergeOccupations(OccsTotalsNH)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


    occupation  6p_total  # of people % of total people
0        baker   2159.04            1              None
1   blacksmith     54.88            1              None
2        clerk    715.60            7              None
3    shoemaker    742.05            2              None
4          esq  61539.89           66              None
5       farmer    778.04            1              None
6    gentleman   3603.06            6              None
7    goldsmith    871.32            1              None
8    carpenter    106.61            2              None
9     marniner    785.62            2              None
10    merchant  29715.41           26              None
11  occupation  48957.43            8              None
12     painter     48.61            1              None
13      doctor   7111.43           18              None
14    spinster     89.13            1              None
15      trader   2807.27            8              None
16   treasurer  

In [47]:
TurnToExcel(OccsTotalsNH, State)

# New Jersey

In [48]:
State = 'NJ'
AllOccsNJ = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNJ = AllOccsNJ.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNJ)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       575.0      62.0          287.0          81.0      155.0      41.0   
1       151.0      40.0           75.0          70.0      119.0       8.0   
2       188.0      14.0           94.0           7.0      210.0       2.0   
3      3676.0      44.0         1838.0          22.0     2106.0      11.0   
4       124.0      21.0           62.0          11.0       56.0      20.0   
5      1324.0      81.0          662.0          41.0      675.0      52.0   
6       844.0      44.0          422.0          23.0      527.0      57.0   
7       161.0      74.0           80.0          87.0       72.0      83.0   
8        91.0      66.0           45.0          84.0       44.0       5.0   
9       750.0      82.0          375.0          42.0      407.0      55.0   
10      524.0      11.0          262.0           6.0      278.0      48.0   
11      131.0      66.0           65.0          84.0       62.0      95.0   

In [49]:
OccsTotalsNJ = AllOccsNJ.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsNJ['# of people'] = OccsTotalsNJ['occupation'].map(AllOccsNJ['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsNJ.to_string())

temp_df = OccsTotalsNJ[['occupation']]

                                                   occupation  6p_total  # of people
0                                             Attorney at law   1325.68            1
1                                                     Esquire      0.00            1
2                        Executor of Michael De Mott deceased   1324.81            1
3                                                      Farmer      3.34            1
4                                                   Gentleman   6063.15            2
5  In trust for Jacobus Van Eis of Somerset County New Jersey     16.67            1
6                                                    Merchant    235.01            1
7                                               No Occupation  19195.17           31
8                                                   Physician    384.34            1


In [50]:
OccsTotalsNJ = RemoveStopwords(OccsTotalsNJ)
print(OccsTotalsNJ.to_string())

temp_df['new occupation'] = OccsTotalsNJ['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

                                            occupation  6p_total  # of people
0                                         attorney law   1325.68            1
1                                              esquire      0.00            1
2                    executor michael de mott deceased   1324.81            1
3                                               farmer      3.34            1
4                                            gentleman   6063.15            2
5  in trust jacobus van eis somerset county new jersey     16.67            1
6                                             merchant    235.01            1
7                                        no occupation  19195.17           31
8                                            physician    384.34            1


/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/1264834632.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsNJ['occupation']


In [51]:
retdict = FindRemoveConst(OccsTotalsNJ['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


attorney law --> attorney law
esquire --> esquire
executor michael de mott deceased --> executor michael de mott deceased
farmer --> farmer
gentleman --> gentleman
in trust jacobus van eis somerset county new jersey --> in trust jacobus van eis somerset county new jersey
merchant --> merchant
no occupation --> no occupation
physician --> physician


In [52]:
Remove = 2

In [53]:
retdict = PrintFutureOccupations(OccsTotalsNJ['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


attorney law --> attorney
esquire --> esquire
executor michael de --> executor
farmer --> farmer
gentleman --> gentleman
in trust jacobus van eis somerset county --> county
merchant --> merchant
no occupation --> occupation
physician --> physician


In [54]:
retdict, OccsTotalsNJ = MergeOccupations(OccsTotalsNJ)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


   occupation  6p_total  # of people % of total people
0    attorney   1325.68            1              None
1         esq      0.00            1              None
2    executor   1324.81            1              None
3      farmer      3.34            1              None
4   gentleman   6063.15            2              None
5    assignee     16.67            1              None
6    merchant    235.01            1              None
7  occupation  19195.17           31              None
8      doctor    384.34            1              None


After Removing Duplicates and Summing
-------------------------------------


   occupation  6p_total  # of people  % of total people
0    assignee     16.67            1                  0
1    attorney   1325.68            1                  0
2      doctor    384.34            1                  0
3         esq      0.00            1                  0
4    executor   1324.81            1                  0
5 

In [55]:
TurnToExcel(OccsTotalsNJ, State)

# New York

In [56]:
State = 'NY'
AllOccsNY = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNY = AllOccsNY.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNY)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       248.0   46.0000          124.0       23.0000      144.0       NaN   
1       738.0   43.0000          369.0       22.0000      708.0     13.00   
2       433.0   28.0000          216.0       64.0000      355.0      8.00   
3        47.0   38.0000           23.0       69.0000       49.0     98.00   
4       131.0   15.0000           65.0       57.0000      146.0     69.00   
5       118.0   45.0000           59.0       23.0000      124.0     94.00   
6        23.0   69.0000           11.0       84.0000       24.0     99.00   
7       261.0   31.0000          130.0       65.0000      340.0     53.00   
8       144.0   55.0000           72.0       28.0000       54.0       NaN   
9       933.0   33.0000          466.0       67.0000      781.0     67.00   
10      555.0   86.0000          277.0       93.0000      637.0     27.00   
11      230.0   90.0000          115.0       45.0000      179.0     91.00   

In [57]:
OccsTotalsNY = AllOccsNY.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsNY['# of people'] = OccsTotalsNY['occupation'].map(AllOccsNY['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsNY.to_string())

temp_df = OccsTotalsNY[['occupation']]

      occupation     6p_total  # of people
0         Broker  1776.120000            1
1         Farmer   674.310000            2
2        Mariner   377.490000            1
3       Merchant  4997.975500           21
4      Merchants  2450.827633            3
5  No Occupation  5897.997733           24
6      Physician   933.330000            1
7   Schoolmaster   131.150000            1


In [58]:
OccsTotalsNY = RemoveStopwords(OccsTotalsNY) 
print(OccsTotalsNY.to_string())

temp_df['new occupation'] = OccsTotalsNY['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

      occupation     6p_total  # of people
0         broker  1776.120000            1
1         farmer   674.310000            2
2        mariner   377.490000            1
3       merchant  4997.975500           21
4      merchants  2450.827633            3
5  no occupation  5897.997733           24
6      physician   933.330000            1
7   schoolmaster   131.150000            1


/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/1653663519.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsNY['occupation']


In [59]:
retdict = PrintFutureOccupations(OccsTotalsNY['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


broker --> broker
farmer --> farmer
mariner --> mariner
merchant --> merchant
merchants --> merchants
no occupation --> occupation
physician --> physician
schoolmaster --> schoolmaster


In [60]:
retdict, OccsTotalsNY = MergeOccupations(OccsTotalsNY)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


     occupation     6p_total  # of people % of total people
0        broker  1776.120000            1              None
1        farmer   674.310000            2              None
2       mariner   377.490000            1              None
3      merchant  4997.975500           21              None
4     merchants  2450.827633            3              None
5    occupation  5897.997733           24              None
6        doctor   933.330000            1              None
7  schoolmaster   131.150000            1              None


After Removing Duplicates and Summing
-------------------------------------


     occupation     6p_total  # of people  % of total people
0        broker  1776.120000            1                  0
1        doctor   933.330000            1                  0
2        farmer   674.310000            2                  0
3       mariner   377.490000            1                  0
4      merchant  4997.975500           21  

In [61]:
TurnToExcel(OccsTotalsNY, State)

# Pennsylvania

In [62]:
State = 'PA'
AllOccsPA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsPA = AllOccsPA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsPA)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0          1.0    965.91            1.0        482.95        1.0   1110.37   
1          1.0    102.14            1.0         51.07        1.0    117.34   
2        158.0      1.00           79.0           NaN      158.0     10.00   
3       3212.0     38.00         1606.0         19.00     1400.0     66.00   
4       9601.0     24.00         4800.0         62.00     3079.0     84.00   
..         ...       ...            ...           ...        ...       ...   
937      582.0     52.00          291.0         26.00       92.0     84.00   
938      150.0       NaN           75.0           NaN      138.0     90.00   
939      526.0     95.00          263.0         47.00      281.0     74.00   
940      143.0     33.00           71.0         66.00      107.0     51.00   
941        NaN       NaN          423.0         14.00        NaN       NaN   

             town state     occupation                   Name  

In [63]:
OccsTotalsPA = AllOccsPA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsPA['# of people'] = OccsTotalsPA['occupation'].map(AllOccsPA['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsPA.to_string())

temp_df = OccsTotalsPA[['occupation']]

                                                         occupation     6p_total  # of people
0                                                        AH.Vat Law      58.0500            1
1        Adm of Alexander McCaraher late of Chester County Deceased     799.5100            1
2                            Administer of the estate of Job Vernon    1865.4400            1
3                                  Administor to the Est of J James      27.5300            1
4                                                   Attorney at Law    1223.6200            1
5                                                       Black Smith     692.2500            1
6                                                       Book Binder     783.0700            1
7                                                            Broker   64008.8400           60
8                                                             Clerk    1855.5800            4
9                                                           

In [64]:
OccsTotalsPA = RemoveStopwords(OccsTotalsPA) 
print(OccsTotalsPA.to_string())

temp_df['new occupation'] = OccsTotalsPA['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

                                               occupation     6p_total  # of people
0                                              ah.vat law      58.0500            1
1    adm alexander mccaraher late chester county deceased     799.5100            1
2                            administer estate job vernon    1865.4400            1
3                                  administor est j james      27.5300            1
4                                            attorney law    1223.6200            1
5                                             black smith     692.2500            1
6                                             book binder     783.0700            1
7                                                  broker   64008.8400           60
8                                                   clerk    1855.5800            4
9                                                      co    2898.8600            4
10                                            conveyancer     108.4000      

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/177176644.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsPA['occupation']


In [65]:
retdict = FindRemoveConst(OccsTotalsPA['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


ah.vat law --> ah.vat law
adm alexander mccaraher late chester county deceased --> adm alexander mccaraher late chester
administer estate job vernon --> administer estate
administor est j james --> administor est
attorney law --> attorney law
black smith --> black smith
book binder --> book binder
broker --> broker
clerk --> clerk
co --> co
conveyancer --> conveyancer
cordwainder --> cordwainder
druggist --> druggist
esquier --> esquier
esquire --> esquire
estate john benezet deceased --> estate john
estate --> estate
ex philadelphia county --> ex
ex william honeyman --> ex
ex estate john baird bucks co --> ex estate john baird
executers david grierlate y town --> executers david grierlate
executor estate abraham hull deceased --> executor estate abraham
executor estate mathias londenberger --> executor estate
executor estate doctor benjamin franklin deceased --> executor estate doctor benjamin
farmer --> f

In [66]:
retdict = PrintFutureOccupations(OccsTotalsPA['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


ah.vat law --> ah.vat
adm alexander mccaraher late chester --> adm
administer estate --> administer
administor est --> administor
attorney law --> attorney
black smith --> black
book binder --> book
broker --> broker
clerk --> clerk
co --> co
conveyancer --> conveyancer
cordwainder --> cordwainder
druggist --> druggist
esquier --> esquier
esquire --> esquire
estate john --> estate
estate --> estate
ex --> ex
ex --> ex
ex estate john baird --> baird
executers david grierlate --> executers
executor estate abraham --> executor
executor estate --> executor
executor estate doctor benjamin --> executor
farmer --> farmer
furrier --> furrier
gentleman --> gentleman
goldsmith --> goldsmith
guardian --> guardian
guardian --> guardian
guardian --> guardian
hatter --> hatter
inn keeper --> inn
junior broker --> junior
law --> law
lawyer --> lawyer
marchant --> marchant
mariner --> mariner
merchant --> mer

In [67]:
retdict, OccsTotalsPA = MergeOccupations(OccsTotalsPA)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


     occupation     6p_total  # of people % of total people
0      attorney      58.0500            1              None
1    administer     799.5100            1              None
2    administer    1865.4400            1              None
3    administor      27.5300            1              None
4      attorney    1223.6200            1              None
5    blacksmith     692.2500            1              None
6   book binder     783.0700            1              None
7        broker   64008.8400           60              None
8         clerk    1855.5800            4              None
9          None    2898.8600            4              None
10  conveyancer     108.4000            1              None
11  cordwainder     281.2500            1              None
12     druggist    4922.4300            2              None
13          esq   20672.0700            6              None
14          esq   20770.1700           15              None
15      

In [68]:
TurnToExcel(OccsTotalsPA, State)

# Rhode Island

In [69]:
State = 'RI'
AllOccsRI = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsRI = AllOccsRI.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsRI)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        154.0      20.0           77.0          10.0      192.0       NaN   
1         60.0      37.0           30.0          18.0       63.0      83.0   
2        342.0      70.0          171.0          35.0      231.0       NaN   
3        194.0      54.0           97.0          27.0      222.0       1.0   
4        130.0      93.0           65.0          47.0       54.0       NaN   
..         ...       ...            ...           ...        ...       ...   
461       28.0      44.0           14.0          22.0       29.0      70.0   
462      142.0      44.0           71.0          22.0      132.0      43.0   
463      183.0      33.0           91.0          67.0      175.0      60.0   
464       83.0      33.0           41.0          67.0       79.0      75.0   
465       33.0      33.0           17.0          39.0       31.0      94.0   

                              town state     occupation  \
0   

In [70]:
OccsTotalsRI = AllOccsRI.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsRI['# of people'] = OccsTotalsRI['occupation'].map(AllOccsRI['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsRI.to_string())

temp_df = OccsTotalsRI[['occupation']]

                                                                          occupation   6p_total  # of people
0                      Administratrix to the Estate of Joshua Brown Esquire Deceased       0.00            1
1                                                                        Auctioneers     460.77            3
2                                                                              Baker     206.37            1
3                                                                         Blacksmith     540.68            3
4                                                                         Blockmaker       4.43            1
5                                                                             Broker     171.50            2
6                                                                            Butcher     235.70            1
7                                                                          Carpenter       0.00            1
8                  

In [71]:
OccsTotalsRI = RemoveStopwords(OccsTotalsRI) 
print(OccsTotalsRI.to_string())

temp_df['new occupation'] = OccsTotalsRI['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

                                                       occupation   6p_total  # of people
0             administratrix estate joshua brown esquire deceased       0.00            1
1                                                     auctioneers     460.77            3
2                                                           baker     206.37            1
3                                                      blacksmith     540.68            3
4                                                      blockmaker       4.43            1
5                                                          broker     171.50            2
6                                                         butcher     235.70            1
7                                                       carpenter       0.00            1
8                                                    chaise maker      47.57            1
9                                                          cooper    1596.15            3
10        

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/3545872157.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsRI['occupation']


In [72]:
retdict = FindRemoveConst(OccsTotalsRI['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


administratrix estate joshua brown esquire deceased --> administratrix estate joshua brown
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
broker --> broker
butcher --> butcher
carpenter --> carpenter
chaise maker --> chaise maker
cooper --> cooper
cordwainer --> cordwainer
deacon --> deacon
distiller --> distiller
doctor --> doctor
esquier --> esquier
esquire --> esquire
extors late jn jenckes esq deceased --> extors late jn jenckes
extors late nicholas brown esq deceased --> extors late nicholas brown
farmer --> farmer
general treasurer state rhode island suceessors said office --> general treasurer state rhode island suceessors
gentleman --> gentleman
glazier --> glazier
goldsmith --> goldsmith
hatter --> hatter
housewright --> housewright
innholder --> innholder
labourer --> labourer
mariner --> mariner
mason --> mason
merchant --> merchant
merchants --> m

In [73]:
Remove = 1

In [74]:
retdict = PrintFutureOccupations(OccsTotalsRI['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administratrix estate joshua brown esquire --> administratrix
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
broker --> broker
butcher --> butcher
carpenter --> carpenter
chaise maker --> chaise
cooper --> cooper
cordwainer --> cordwainer
deacon --> deacon
distiller --> distiller
doctor --> doctor
esquier --> esquier
esquire --> esquire
extors late jn jenckes esq --> extors
extors late nicholas brown esq --> extors
farmer --> farmer
general treasurer state rhode island suceessors said --> general
gentleman --> gentleman
glazier --> glazier
goldsmith --> goldsmith
hatter --> hatter
housewright --> housewright
innholder --> innholder
labourer --> labourer
mariner --> mariner
mason --> mason
merchant --> merchant
merchants --> merchants
no occupation --> occupation
physician --> physician
reverend --> reverend
shipwright --> shipwright
shoemaker --

In [75]:
retdict, OccsTotalsRI = MergeOccupations(OccsTotalsRI)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


        occupation   6p_total  # of people % of total people
0   administratrix       0.00            1              None
1      auctioneers     460.77            3              None
2            baker     206.37            1              None
3       blacksmith     540.68            3              None
4       blockmaker       4.43            1              None
5           broker     171.50            2              None
6          butcher     235.70            1              None
7        carpenter       0.00            1              None
8     chaise maker      47.57            1              None
9           cooper    1596.15            3              None
10       shoemaker     417.48            1              None
11          deacon      16.35            1              None
12       distiller     466.25            3              None
13          doctor     445.45            1              None
14             esq      13.08            1           

In [76]:
TurnToExcel(OccsTotalsRI, State)

# South Carolina

In [77]:
State = 'SC'
AllOccsSC = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsSC = AllOccsSC.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsSC)

     6p_Dollar   6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0          1.0  1580.6300            1.0      790.3100        1.0   1818.62   
1          1.0    47.4800            1.0       23.7500        1.0     51.90   
2          1.0   824.4133            1.0      412.2033        1.0    941.54   
3          1.0   322.7433            1.0      161.3733        1.0    349.98   
4          1.0   333.0800            1.0      166.5400        1.0    359.55   
..         ...        ...            ...           ...        ...       ...   
357        NaN        NaN            NaN           NaN      167.0     77.00   
358        NaN        NaN            NaN           NaN     3600.0       NaN   
359       84.0    13.0000           42.0        7.0000       88.0     48.00   
360      242.0    67.0000          121.0       33.0000       79.0     52.00   
361      180.0    47.0000           90.0       24.0000      207.0     93.00   

                           town state       occupat

In [78]:
OccsTotalsSC = AllOccsSC.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsSC['# of people'] = OccsTotalsSC['occupation'].map(AllOccsSC['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsSC.to_string())

temp_df = OccsTotalsSC[['occupation']]

                                      occupation       6p_total  # of people
0            Administrator to John Harleston Jun      33.330000            1
1                 Administrator to Joseph Warley    1671.440000            1
2                       Assignee of James Simons     345.020000            1
3                                Attorney at Law    3690.940000            9
4                                      Autioneer      10.650000            1
5                                   Blacks smith    1423.480000            1
6                                     Blockmaker    5264.230000            2
7                                     Bricklayer      37.350000            1
8                                         Broker    2334.890000           11
9                                  Cabinet Maker      74.460000            1
10                                     Carpenter     449.020000            5
11                          Charleston Merchants      26.670000            2

In [79]:
OccsTotalsSC = RemoveStopwords(OccsTotalsSC) 
print(OccsTotalsSC.to_string())

temp_df['new occupation'] = OccsTotalsSC['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

                           occupation       6p_total  # of people
0    administrator john harleston jun      33.330000            1
1         administrator joseph warley    1671.440000            1
2               assignee james simons     345.020000            1
3                        attorney law    3690.940000            9
4                           autioneer      10.650000            1
5                        blacks smith    1423.480000            1
6                          blockmaker    5264.230000            2
7                          bricklayer      37.350000            1
8                              broker    2334.890000           11
9                       cabinet maker      74.460000            1
10                          carpenter     449.020000            5
11               charleston merchants      26.670000            2
12                       clerk senate    1425.820000            2
13                 comtroller customs      53.740000            1
14        

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/2747012364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsSC['occupation']


In [80]:
retdict = FindRemoveConst(OccsTotalsSC['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


administrator john harleston jun --> administrator john harleston
administrator joseph warley --> administrator joseph
assignee james simons --> assignee james
attorney law --> attorney law
autioneer --> autioneer
blacks smith --> blacks smith
blockmaker --> blockmaker
bricklayer --> bricklayer
broker --> broker
cabinet maker --> cabinet maker
carpenter --> carpenter
charleston merchants --> charleston merchants
clerk senate --> clerk senate
comtroller customs --> comtroller customs
esq --> esq
excuetor --> excuetor
executor philip hawkins --> executor philip
executor benjamin tucker --> executor benjamin
executor jacob smiser --> executor jacob
executor james peart --> executor james
executor samuel bonneau --> executor samuel
executor sarah lampriere --> executor sarah
executor william gibbs --> executor william
executor william parker --> executor william
executor richard mason --> executor richard
execu

In [81]:
Remove = 1

In [82]:
retdict = PrintFutureOccupations(OccsTotalsSC['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administrator john harleston --> administrator
administrator joseph --> administrator
assignee james --> assignee
attorney law --> attorney
autioneer --> autioneer
blacks smith --> blacks
blockmaker --> blockmaker
bricklayer --> bricklayer
broker --> broker
cabinet maker --> cabinet
carpenter --> carpenter
charleston merchants --> charleston
clerk senate --> clerk
comtroller customs --> comtroller
esq --> esq
excuetor --> excuetor
executor philip --> executor
executor benjamin --> executor
executor jacob --> executor
executor james --> executor
executor samuel --> executor
executor sarah --> executor
executor william --> executor
executor william --> executor
executor richard --> executor
executrix william --> executrix
factor --> factor
gentleman --> gentleman
hair dresser --> hair
mariner --> mariner
marriner --> marriner
merchant --> merchant
merchant executor benjamin --> merchant
merchant

In [83]:
retdict, OccsTotalsSC = MergeOccupations(OccsTotalsSC)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


        occupation       6p_total  # of people % of total people
0    administrator      33.330000            1              None
1    administrator    1671.440000            1              None
2         assignee     345.020000            1              None
3         attorney    3690.940000            9              None
4        autioneer      10.650000            1              None
5       blacksmith    1423.480000            1              None
6       blockmaker    5264.230000            2              None
7       bricklayer      37.350000            1              None
8           broker    2334.890000           11              None
9    cabinet maker      74.460000            1              None
10       carpenter     449.020000            5              None
11       merchants      26.670000            2              None
12           clerk    1425.820000            2              None
13      comtroller      53.740000            1            

In [84]:
TurnToExcel(OccsTotalsSC, State)

# Virginia

In [85]:
State = 'VA'
AllOccsVA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsVA = AllOccsVA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsVA)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        17.0  351.6233           13.0      175.8133       14.0    384.09   
1       910.0   50.0000          455.0       25.0000      962.0     84.00   
2       834.0   18.0000          417.0        8.0000      895.0     10.00   
3      1936.0   28.0000          968.0       14.0000        NaN       NaN   
4       243.0   94.0000          121.0       96.0000       65.0     85.00   
5      1034.0   12.0000          517.0        6.0000      558.0     42.00   
6       165.0   92.0000           82.0       96.0000      176.0     23.00   
7       369.0   70.0000          184.0       84.0000      360.0     94.00   
8      1468.0   80.0000          734.0       40.0000     1807.0     86.00   
9      2363.0   94.0000         1181.0       97.0000     1804.0     50.00   
10      309.0   60.0000          154.0       80.0000      167.0     17.00   
11      165.0   89.0000           82.0       94.0000      666.0     21.00   

In [86]:
OccsTotalsVA = AllOccsVA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsVA['# of people'] = OccsTotalsVA['occupation'].map(AllOccsVA['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsVA.to_string())

temp_df = OccsTotalsVA[['occupation']]

      occupation      6p_total  # of people
0      Gentleman     78.330000            1
1       Merchant     70.480000            2
2      Merchants     20.516233            1
3  No Occupation  17949.610000           28
4         Weaver     46.020000            1


In [87]:
OccsTotalsVA = RemoveStopwords(OccsTotalsVA) 
print(OccsTotalsVA.to_string())

temp_df['new occupation'] = OccsTotalsVA['occupation']
ogoccs = pd.concat([ogoccs, temp_df])

      occupation      6p_total  # of people
0      gentleman     78.330000            1
1       merchant     70.480000            2
2      merchants     20.516233            1
3  no occupation  17949.610000           28
4         weaver     46.020000            1


/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_60320/3576601614.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['new occupation'] = OccsTotalsVA['occupation']


In [88]:
retdict = FindRemoveConst(OccsTotalsVA['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupation Before --> Occupation After
--------------------------------------


gentleman --> gentleman
merchant --> merchant
merchants --> merchants
no occupation --> no occupation
weaver --> weaver


In [89]:
retdict = PrintFutureOccupations(OccsTotalsVA['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


gentleman --> gentleman
merchant --> merchant
merchants --> merchants
no occupation --> occupation
weaver --> weaver


In [90]:
retdict, OccsTotalsVA = MergeOccupations(OccsTotalsVA)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


   occupation      6p_total  # of people % of total people
0   gentleman     78.330000            1              None
1    merchant     70.480000            2              None
2   merchants     20.516233            1              None
3  occupation  17949.610000           28              None
4      weaver     46.020000            1              None


After Removing Duplicates and Summing
-------------------------------------


   occupation      6p_total  # of people  % of total people
0   gentleman     78.330000            1                  0
1    merchant     70.480000            2                  0
2   merchants     20.516233            1                  0
3  occupation  17949.610000           28                  0
4      weaver     46.020000            1                  0


Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


   occupation      6p_total  # of people  % of total people  average

In [91]:
TurnToExcel(OccsTotalsVA, State)

# All States

In [92]:
#Append each states occupation + totals dataframes to the end of each other. 
OccsTotalsAll = pd.concat([OccsTotalsCT, OccsTotalsGA, OccsTotalsMD, OccsTotalsNC, OccsTotalsNH, OccsTotalsNJ,
                          OccsTotalsNY, OccsTotalsPA, OccsTotalsRI, OccsTotalsSC, OccsTotalsVA]).reset_index(drop=True)
print(OccsTotalsAll.to_string())

            occupation       6p_total  # of people  % of total people  average debt per person
0        no occupation  673180.800000          432          47.213115              1558.288889
1             merchant   63765.000000           63           6.885246              1012.142857
2               farmer   36780.360000          170          18.579235               216.355059
3            gentleman   16624.320000           32           3.497268               519.510000
4            treasurer   15387.150000           14           1.530055              1099.082143
5              esquire   13020.090000           13           1.420765              1001.545385
6             attorney   11114.600000           20           2.185792               555.730000
7               trader    9454.130000           46           5.027322               205.524565
8                widow    6488.110000           28           3.060109               231.718214
9             executor    4558.250000            7

In [93]:
OccsTotalsAll = OccsTotalsAll.groupby('occupation')[['6p_total', '# of people']].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsAll.to_string())

           occupation      6p_total  # of people
0       administrator  8.500810e+03         17.0
1      administratrix  0.000000e+00          1.0
2          apothecary  7.720000e+00          1.0
3            assignee  2.416330e+03          4.0
4            attorney  2.482790e+04         35.0
5          auctioneer  1.231900e+02          2.0
6         auctioneers  4.607700e+02          3.0
7               baker  2.365410e+03          2.0
8          blacksmith  3.436000e+03          9.0
9          blockmaker  5.268660e+03          3.0
10        book binder  7.830700e+02          1.0
11         bricklayer  3.735000e+01          1.0
12             broker  7.919102e+04         80.0
13            butcher  2.357000e+02          1.0
14      cabinet maker  7.446000e+01          1.0
15          carpenter  7.843000e+02         10.0
16            cashier  8.530000e+01          2.0
17       chaise maker  4.757000e+01          1.0
18              clerk  3.997000e+03         13.0
19           clothie

In [94]:
retdict = PrintFutureOccupations(OccsTotalsAll['occupation'].tolist())
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administrator --> administrator
administratrix --> administratrix
apothecary --> apothecary
assignee --> assignee
attorney --> attorney
auctioneer --> auctioneer
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
book binder --> book
bricklayer --> bricklayer
broker --> broker
butcher --> butcher
cabinet maker --> cabinet
carpenter --> carpenter
cashier --> cashier
chaise maker --> chaise
clerk --> clerk
clothier --> clothier
committee member --> committee
comtroller --> comtroller
conveyancer --> conveyancer
cooper --> cooper
cordwainer --> cordwainer
deacon --> deacon
distiller --> distiller
doctor --> doctor
doctors --> doctors
druggist --> druggist
esquire --> esquire
estate --> estate
executor --> executor
executors --> executors
extors --> extors
farmer --> farmer
frederick company --> frederick
furrier --> furrier
gentleman --> gentleman
glaz

In [95]:
#Exclude these wors from any fuzzy string matching. Causes errors. 
NoMerge.append('planter')
NoMerge.append('printer')

In [96]:
retdict, OccsTotalsAll = MergeOccupations(OccsTotalsAll)
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: retdict.get(x, x))

After Cleaning
--------------


           occupation      6p_total  # of people % of total people
0       administrator  8.500810e+03         17.0              None
1      administratrix  0.000000e+00          1.0              None
2          apothecary  7.720000e+00          1.0              None
3            assignee  2.416330e+03          4.0              None
4            attorney  2.482790e+04         35.0              None
5          auctioneer  1.231900e+02          2.0              None
6         auctioneers  4.607700e+02          3.0              None
7               baker  2.365410e+03          2.0              None
8          blacksmith  3.436000e+03          9.0              None
9          blockmaker  5.268660e+03          3.0              None
10        book binder  7.830700e+02          1.0              None
11         bricklayer  3.735000e+01          1.0              None
12             broker  7.919102e+04         80.0              None
13            butcher  2.35700

In [97]:
#Remove any occupatins will a total of '0'
OccsTotalsAll = OccsTotalsAll[OccsTotalsAll['6p_total'] != 0].reset_index(drop=True)
OccsTotalsAll['% of total people'] = (OccsTotalsAll['# of people'] / OccsTotalsAll['# of people'].sum()) * 100 #Recalculate the '% of total people'.
print(OccsTotalsAll.to_string())

           occupation      6p_total  # of people  % of total people  average debt per person
0          occupation  2.980604e+06         1662          46.450531              1793.383813
1            marchant  9.751808e+05          668          18.669648              1459.851506
2           treasurer  2.514448e+05           32           0.894354              7857.650625
3                 esq  1.845736e+05          235           6.567915               785.419566
4              broker  7.919102e+04           80           2.235886               989.887750
5              farmer  5.914360e+04          274           7.657909               215.852555
6           gentleman  4.136731e+04           60           1.676914               689.455167
7               widow  3.194785e+04           80           2.235886               399.348125
8              doctor  2.524972e+04           44           1.229737               573.857273
9            attorney  2.482790e+04           35           0.978200   

In [98]:
TurnToExcel(OccsTotalsAll, 'AllStates')

In [99]:
ogoccs['new occupation'] = ogoccs['new occupation'].apply(lambda x: datadict.get(x, x))
ogoccs.drop_duplicates().to_csv('occupation_crosswalk.csv')